## Practicum 2: Rohin Arora, Kiran Tulsulkar, Varun S, Aditya Patgaonkar

## Q1
Data inspection has been done using another notebook attached with the submission: PracticumDataInspection.ipynb

In [89]:
# Preprocess data
'''
    Replace "," in MotionPitcure.csv -- originalTitle and crew
    with " " to avoid delimiter issues in the CSV
'''
mpcsv=pd.read_csv("FinalTables/MotionPitcure.csv")
mpcsv['originalTitle']=mpcsv['originalTitle'].str.replace(","," ")
mpcsv['crew']=mpcsv['crew'].str.replace(","," ")
mpcsv.to_csv("FinalTables/MotionPitcure.csv", index=False)

/Users/rohinarora/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
'''
    Replace "," in FinalTables.csv -- personName
    with " " to avoid delimiter issues in the CSV
'''

crewcsv=pd.read_csv("FinalTables/Crew.csv")
crewcsv['personName']=crewcsv['personName'].str.replace(","," ")
crewcsv.to_csv("FinalTables/Crew.csv",index=False)

In [83]:
'''
    Replace "," in Title.csv -- title
    with " " to avoid delimiter issues in the CSV
'''

Titlecsv=pd.read_csv("FinalTables/Title.csv")
Titlecsv['title']=Titlecsv['title'].str.replace(","," ")
Titlecsv.to_csv("FinalTables/Title.csv",index=False)

/Users/rohinarora/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 2A ERD -- Crow's foot notation
<img src="images/2Aup.png">

### 2B -- Normalized ERD notation
<img src="images/2Bup.png">

### 2C -- Added attributes
Added age and numMovies in Crew (which we'll be updating later with UPDATE statements)
<img src="images/2Cupdated.png">

### 2D
> Why to denormalize a database:
1. To enhance query performance
2. To make a database more convenient to manage
3. To facilitate and accelerate reporting

### Test connection

In [1]:
import mysql.connector
import pandas as pd
from mysql.connector import errorcode
def testSql():
    ''' 
    Test SQL database connection, cursor and test it 
    '''
    # Connect to server Mysql@localhost:3306
    cnx = mysql.connector.connect(
        host="127.0.0.1",
        port=3306,
        user="root",
        password="12345678")

    # Get a cursor
    cur = cnx.cursor()

    # Execute a query
    cur.execute("SELECT CURDATE()")

    # Fetch one result
    row = cur.fetchone()
    print("Current date is: {0}".format(row[0]))

    # Close connection
    cnx.close()

testSql()

Current date is: 2020-11-16


## Q3 and Q4
### Create Tables and load data

In [2]:
''' Intialize SQL database connection, cursor '''
# connect to server
cnx = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="12345678")

# Get a cursor
cursor = cnx.cursor()

In [3]:
# create database
DB_NAME = 'IMDB'
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
    print ("DB exists")
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

DB exists


In [4]:
TABLES = {}

#### Create MotionPicture Table

In [102]:
TABLES['MotionPicture'] = (
    """
CREATE TABLE `MotionPicture` (
  `mpId` INT NOT NULL PRIMARY KEY,
  `titleType` VARCHAR(300),
  `primaryTitle` VARCHAR(300),
  `isAdult` Boolean,
  `startYear` INT,
  `endYear` INT,
  `runtimeMinutes` INT,
  `crewList` VARCHAR(400)
)
    """
    )

#create table
table_name='MotionPicture'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")
        
        
# insert data into MotionPitcure table
add_MotionPitcure = ("""INSERT INTO MotionPicture 
(mpId, titleType,primaryTitle,isAdult,startYear,endYear,runtimeMinutes,crewList) 
VALUES (%s, %s,%s, %s,%s, %s,%s,%s)
""")

'''
Since some mpIds in the MotionPitcure table 
are not referenced in Episode, Rating, Genre, Crew,
we are keeping track of those Ids while creating the 
tables which reference mpId as FOREIGN KEY to avoid conflicts.
'''
ignoreMPID=[]
with open("FinalTables/MotionPitcure.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        crewInfo=data[-1]
        try:
            cursor.execute(add_MotionPitcure,(data[0],data[1],data[2],data[3],data[4],data[5],data[6],crewInfo))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            if "Out of range value for column 'isAdult' at row" in str(err) or "Data too long for column 'primaryTitle'" in str(err) :
                ignoreMPID.append(data[0])
                continue
            print (err)
            break
cnx.commit()

Creating table MotionPicture: already exists.


In [50]:
# Get number of records in MotionPitcure table
cursor.execute("select count(*) from MotionPicture")
cursor.fetchone() #total counts in DB

(7316894,)

#### Create Episode table

In [11]:
TABLES['Episode'] = (
    """
CREATE TABLE `Episode` (
  `episodeId` INT NOT NULL PRIMARY KEY,
  `seasonNumber` INT,
  `episodeNumber` INT,
  `mpId` INT,
  FOREIGN KEY(mpId) REFERENCES MotionPicture(mpId)
)
    """
    )
table_name='Episode'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")
        
        
add_Episode = ("""INSERT INTO Episode 
(episodeId, seasonNumber,episodeNumber,mpId) 
VALUES (%s, %s,%s,%s)
""")

'''
Since some mpIds in the MotionPitcure table 
are not referenced in Episode we are keeping 
track of those Ids while creating the 
tables which reference mpId as FOREIGN KEY to avoid conflicts.
'''
with open("FinalTables/Episode.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        if data[3] in ignoreMPID: continue
        try:
            cursor.execute(add_Episode,(data[0],data[1],data[2],data[3]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            if  "Cannot add or update a child row: a foreign key constraint fails" in str(err): continue
            print (err)
            break
cnx.commit()


Creating table Episode: already exists.


In [49]:
cursor.execute("select count(*) from Episode")
cursor.fetchone() #total counts in DB

(5275338,)

#### Create Rating Table

In [43]:
TABLES['Rating'] = (
    """
CREATE TABLE `Rating` (
`ratingId` INT NOT NULL PRIMARY KEY,
`averageRating` FLOAT,
`numVotes` INT,
`mpId` INT,
FOREIGN KEY(mpId) REFERENCES MotionPicture(mpId)
)
    """
)

table_name='Rating'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")
    
    
add_rating = ("""INSERT INTO rating 
(ratingId, averageRating,numVotes,mpId) 
VALUES (%s, %s,%s,%s)
""")
with open("FinalTables/Rating.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue
        data=row.strip().split(',')
        if data[3] in ignoreMPID: continue
        try:
            cursor.execute(add_rating,(data[0],data[1],data[2],data[3]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            print (err)
cnx.commit()

Creating table Rating: already exists.


In [47]:
cursor.execute("select count(*) from rating")
cursor.fetchone() #total counts in DB

(1088151,)

#### Create Genre Table

In [46]:
TABLES['genre'] = (
"""
CREATE TABLE `Genre` (
  `genreId` INT NOT NULL PRIMARY KEY,
  `name` VARCHAR(300)
)
""")


table_name='genre'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre


add_genre = ("""INSERT INTO genre 
(genreId, name) 
VALUES (%s, %s)
""")
with open("FinalTables/Genre.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        try:
            cursor.execute(add_genre,(data[0],data[1]))
        except mysql.connector.Error as err:
            print (err)
cnx.commit()

Creating table genre: OK


In [48]:
cursor.execute("select count(*) from genre")
cursor.fetchone()  #total counts in DB

(28,)

#### Create Role Table

In [52]:
TABLES['Role'] = (
"""
CREATE TABLE `Role` (
  `roleId` INT NOT NULL PRIMARY KEY,
  `name` VARCHAR(500)
)
""")

table_name='Role'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre

    
add_Role = ("""INSERT INTO Role 
(roleId, name) 
VALUES (%s, %s)
""")
with open("FinalTables/Role.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        try:
            cursor.execute(add_Role,(data[0],data[1]))
        except mysql.connector.Error as err:
            print (err)
cnx.commit()

Creating table Role: already exists.


In [53]:
cursor.execute("select count(*) from Role")
cursor.fetchone() #total counts in DB

(43,)

####  Create Crew Table

In [63]:
TABLES['Crew'] = (
"""
CREATE TABLE `Crew` (
  `crewId` INT NOT NULL PRIMARY KEY,
  `personName` VARCHAR(500),
  `birthYear` INT,
  `deathYear` INT,
  `status` INT,
  `age` INT,
  `numMovies` INT
)
""")

table_name='Crew'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre
    

add_Crew = ("""INSERT INTO Crew 
(crewId, personName,birthYear,deathYear,status) 
VALUES (%s, %s,%s,%s,%s)
""")
with open("FinalTables/Crew.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        try:
            cursor.execute(add_Crew,(data[0],data[1],data[2],data[3],data[4]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            print (err)
            break
cnx.commit()

Creating table Crew: already exists.


In [64]:
cursor.execute("select count(*) from Crew")
cursor.fetchone() #total counts in DB

(10486504,)

#### Create RoleCrew

In [69]:
TABLES['RoleCrew'] = (
"""
CREATE TABLE `RoleCrew` (
  `rcId` INT NOT NULL PRIMARY KEY AUTO_INCREMENT,
  `crewId` INT,
  `roleId` INT,
  FOREIGN KEY(crewId) REFERENCES Crew(crewId),
  FOREIGN KEY(roleId) REFERENCES Role(roleId)
)

""")


table_name='RoleCrew'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre
    

ignorecrewId=set()
add_RoleCrew = ("""INSERT INTO RoleCrew 
(rcId, crewId,roleId) 
VALUES (%s, %s,%s)
""")
with open("FinalTables/RoleCrew.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        try:
            cursor.execute(add_RoleCrew,(data[0],data[1],data[2]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            if "Cannot add or update a child row: a foreign key constraint fails (`imdb`.`rolecrew`, CONSTRAINT `rolecrew_ibfk_1` FOREIGN KEY (`crewId`) REFERENCES `crew` (`crewId`))" in str(err): 
                ignorecrewId.add(data[1])
                continue
            print (err)
            break
cnx.commit()

Creating table RoleCrew: already exists.


In [70]:
cursor.execute("select count(*) from RoleCrew")
cursor.fetchone() #total counts in DB

(41775514,)

#### Create MotionPictureCrew

In [88]:
TABLES['MotionPictureCrew'] = (
"""
CREATE TABLE `MotionPictureCrew` (
  `mpcId` INT NOT NULL PRIMARY KEY AUTO_INCREMENT,
  `mpId` INT,
  `crewId` INT,
  FOREIGN KEY(crewId) REFERENCES Crew(crewId),
  FOREIGN KEY(mpId) REFERENCES MotionPicture(mpId)
)
""")


table_name='MotionPictureCrew'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre
    

ignorempcrewId=set()
ignorempcrewId2=set()
add_MotionPictureCrew = ("""INSERT INTO MotionPictureCrew 
(mpcId, mpId,crewId) 
VALUES (%s, %s,%s)
""")

with open("FinalTables/MotionPitcureCrew.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        if data[1] in ignoreMPID: continue
        try:
            cursor.execute(add_MotionPictureCrew,(data[0],data[1],data[2]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            if "Cannot add or update a child row: a foreign key constraint fails (`imdb`.`motionpicturecrew`, CONSTRAINT `motionpicturecrew_ibfk_1` FOREIGN KEY (`crewId`) REFERENCES `crew` (`crewId`))" in str(err): 
                ignorempcrewId.add(data[2])
                continue
            if "Cannot add or update a child row: a foreign key constraint fails (`imdb`.`motionpicturecrew`, CONSTRAINT `motionpicturecrew_ibfk_2` FOREIGN KEY (`mpId`) REFERENCES `motionpicture` (`mpId`))" in str(err): 
                ignorempcrewId2.add(data[2])
                continue
            print (err)
            break
cnx.commit()


Creating table MotionPictureCrew: already exists.
1206 (HY000): The total number of locks exceeds the lock table size


### P.S: Due to the above error MotionPitcureCrew could not be loaded fully. Hence Views from Q8 could not be executed.

#### Create Title

In [95]:
TABLES['Title'] = (
"""
CREATE TABLE `Title` (
  `titleId` INT NOT NULL PRIMARY KEY,
  `name` VARCHAR(500),
  `region` VARCHAR(20),
  `language` VARCHAR(20),
  `isOriginalTitle` Boolean,
  `mpId` INT,
  FOREIGN KEY(mpId) REFERENCES MotionPicture(mpId)
)
""")


table_name='Title'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre



add_Title = ("""INSERT INTO Title 
(titleId, name,region,language,isOriginalTitle,mpId) 
VALUES (%s, %s,%s, %s,%s, %s)
""")
run=10
with open("FinalTables/Title.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        if data[5] in ignoreMPID: continue
        try:
            cursor.execute(add_Title,(data[0],data[1],data[2],data[3],data[4],data[5]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            if "Incorrect integer value: '\\N' for column 'isOriginalTitle' at row 1" in str(err): continue
            print (err)
            break
        run-=1
        if run==0: break
cnx.commit()

Creating table Title: already exists.


In [96]:
cursor.execute("select count(*) from Title")
cursor.fetchone() #total counts in DB

(141172,)

#### Create MotionPictureGenre

In [99]:
TABLES['MotionPictureGenre'] = (
"""
CREATE TABLE `MotionPictureGenre` (
  `mpgId` INT NOT NULL PRIMARY KEY AUTO_INCREMENT,
  `mpId` INT,
  `genreId` INT,
  FOREIGN KEY(genreId) REFERENCES genre(genreId),
  FOREIGN KEY(mpId) REFERENCES MotionPicture(mpId)
)
""")


table_name='MotionPictureGenre'
table_description = TABLES[table_name]
try:
    print("Creating table {}: ".format(table_name), end='')
    cursor.execute(table_description)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")#### Table Genre
    
    
add_MotionPictureGenre = ("""INSERT INTO MotionPictureGenre 
(mpgId, mpId,genreId) 
VALUES (%s, %s,%s)
""")
run=10000
with open("FinalTables/MotionPitcureGenre.csv", 'r') as csv_data:
    for idx,row in enumerate(csv_data):
        if idx==0: continue #skip header
        data=row.strip().split(',')
        if data[1] in ignoreMPID: continue
        try:
            cursor.execute(add_MotionPictureGenre,(data[0],data[1],data[2]))
        except mysql.connector.Error as err:
            if "Duplicate entry" in str(err): continue
            if "Cannot add or update a child row: a foreign key constraint fails (`imdb`.`motionpicturegenre`, CONSTRAINT `motionpicturegenre_ibfk_1` FOREIGN KEY (`genreId`) REFERENCES `genre` (`genreId`))" in str(err): continue
            print (err)
            break
        run-=1
        if run==0: break
cnx.commit()

Creating table MotionPictureGenre: already exists.


In [ ]:
cursor.execute("select count(*) from MotionPictureGenre")
cursor.fetchone() #total counts in DB

### Referential Integrity Checks

To maintain referential integrity, it is required that a foreign key must have a matching primary key. So, to maintain connectivity between two tables this constraint is specified. To avoid having records which do not point to a table we need to enforce referential integrity that is where the PK is located. When referential integrity is enforced it is important that PK and FK have same data types and come from the same domain. So, while loading it is checked whether PK and FK are from same domain, for join to take place. So before loading we must make sure that the FK for a table is present in the Table for which it is PK. We can also create index to improve performance of checking referential integrity. We can create Indexes for Foreign keys to improve the performance of checks on parent table.

### Q5 UPDATE statements

Here, we have added status (dead/alive). However, according to your instructions in Recitation, we have not used it for querying and updating tables ("age") for creating a view in Q7 

#### Crew table update for age and number of movies

In [101]:
#query=("""DESCRIBE Crew""")
query=("""SHOW COLUMNS FROM Crew""")
cursor.execute(query)
cursor.fetchall()

[('crewId', b'int', 'NO', 'PRI', None, ''),
 ('personName', b'varchar(500)', 'YES', '', None, ''),
 ('birthYear', b'int', 'YES', '', None, ''),
 ('deathYear', b'int', 'YES', '', None, ''),
 ('status', b'int', 'YES', '', None, ''),
 ('age', b'int', 'YES', '', None, ''),
 ('numMovies', b'int', 'YES', '', None, '')]

#### Crew Table before update

In [105]:
query=('''SELECT * from Crew LIMIT 10''')
cursor.execute(query)
cursor.fetchall()

[(1, 'Fred Astaire', 1899, 1987, 0, 0, None),
 (2, 'Lauren Bacall', 1924, 2014, 0, 0, None),
 (3, 'Brigitte Bardot', 1934, 0, 1, 0, None),
 (4, 'John Belushi', 1949, 1982, 0, 0, None),
 (5, 'Ingmar Bergman', 1918, 2007, 0, 0, None),
 (6, 'Ingrid Bergman', 1915, 1982, 0, 0, None),
 (7, 'Humphrey Bogart', 1899, 1957, 0, 0, None),
 (8, 'Marlon Brando', 1924, 2004, 0, 0, None),
 (9, 'Richard Burton', 1925, 1984, 0, 0, None),
 (10, 'James Cagney', 1899, 1986, 0, 0, None)]

#### UPDATE age

In [106]:
query=('''UPDATE Crew SET age = deathYear - birthYear
where deathYear != 0''')
cursor.execute(query)

In [107]:
query=('''UPDATE crew SET age = 2020 - birthYear
where deathYear = 0;''')
cursor.execute(query)

#### Crew Table after update

In [108]:
query=('''SELECT * from Crew LIMIT 10''')
cursor.execute(query)
cursor.fetchall()

[(1, 'Fred Astaire', 1899, 1987, 0, 88, None),
 (2, 'Lauren Bacall', 1924, 2014, 0, 90, None),
 (3, 'Brigitte Bardot', 1934, 0, 1, 86, None),
 (4, 'John Belushi', 1949, 1982, 0, 33, None),
 (5, 'Ingmar Bergman', 1918, 2007, 0, 89, None),
 (6, 'Ingrid Bergman', 1915, 1982, 0, 67, None),
 (7, 'Humphrey Bogart', 1899, 1957, 0, 58, None),
 (8, 'Marlon Brando', 1924, 2004, 0, 80, None),
 (9, 'Richard Burton', 1925, 1984, 0, 59, None),
 (10, 'James Cagney', 1899, 1986, 0, 87, None)]

#### Update numMovies in Crew table

Updation using python's cursor is taking a lot of time and CPU resources. 
That is why we are adding the output on MySQL Workbench using dummy data.

In [ ]:
query=('''UPDATE CREW C
Set numMovies = (select count(*) from motionpicturecrew mpc group by mpc.crewId
                having mpc.crewId = C.crewId);''')
cursor.execute(query)

<img src="images/Update_statement.png">

In [6]:
query=('''SELECT * from Crew LIMIT 10''')
cursor.execute(query)
cursor.fetchall()

[(1, 'Fred Astaire', 1899, 1987, 0, None, None),
 (2, 'Lauren Bacall', 1924, 2014, 0, None, None),
 (3, 'Brigitte Bardot', 1934, 0, 1, None, None),
 (4, 'John Belushi', 1949, 1982, 0, None, None),
 (5, 'Ingmar Bergman', 1918, 2007, 0, None, None),
 (6, 'Ingrid Bergman', 1915, 1982, 0, None, None),
 (7, 'Humphrey Bogart', 1899, 1957, 0, None, None),
 (8, 'Marlon Brando', 1924, 2004, 0, None, None),
 (9, 'Richard Burton', 1925, 1984, 0, None, None),
 (10, 'James Cagney', 1899, 1986, 0, None, None)]

In [4]:
query=('''SELECT * from motionpicturecrew LIMIT 10''')
cursor.execute(query)
cursor.fetchall()

[(1, 1, 1588970),
 (2, 1, 5690),
 (3, 1, 374658),
 (4, 2, 721526),
 (5, 2, 1335271),
 (6, 3, 721526),
 (7, 3, 5442194),
 (8, 3, 1335271),
 (9, 3, 5442200),
 (10, 4, 721526)]

### Q6 Triggers

Triggers below set the age and number of movies for new update on CREW table. The outputs posted below are screenshots taken from another notebook which we have attached with the submission

In [ ]:
query=('''
CREATE TRIGGER `update_age_insert` 
BEFORE INSERT
ON CREW FOR EACH ROW
BEGIN
IF NEW.deathYear != 0 THEN
SET NEW.age = new.deathYear - new.birthYear;
ELSEIF NEW.deathYear = 0 THEN
SET NEW.age = 2020 - new.birthYear;
END IF;
END;
''')
cursor.execute(query)

In [ ]:
query=('''CREATE TRIGGER `update_numovies_insert` 
AFTER INSERT
ON motionpicturecrew FOR EACH ROW
BEGIN
UPDATE CREW C
Set numMovies = (select count(*) from motionpicturecrew mpc
                group by mpc.crewId
                having mpc.crewId = C.crewId);
END;
''')
cursor.execute(query)

<img src="images/Trigger1.png">

#### Q7. View
Name, age, actor, age, status(dead/alive) and number of movies they are known for. 
Due to the number of joins involved it is taking a long time to complete execution using Python. Hence, we are adding test on dummy data using MySQL Workbench.

In [ ]:
query=('''CREATE VIEW ACTOR_INFO AS
SELECT personName,name as role, age, status, count(*) as top_movies from Crew c
inner join RoleCrew rc on rc.rcId = c.crewId
inner join Role r on r.roleId = rc.rcId
inner join MotionPictureCrew mpc on mpc.mpcId = c.crewId
inner join MotionPicture mp on mp.mpId = mpc.mpcId
inner join rating rt on rt.mpId = mp.mpId
where rt.averageRating > 6
group by c.crewId
having r.name = 'actress';''')
cursor.execute(query)

<img src="images/View.png">

#### Q8. Query that finds the number of seasons for each TV series.

In [ ]:
query=('''
select primaryTitle, MAX(seasonNumber) from motionpicture mp
inner join episode e on e.mpId = mp.mpId
group by e.mpId;''')
cursor.execute(query)


<img src="images/Q8ans.png">

<img src="images/freq_plot.png">

#### Q9. addActor() 
Here we send the parameters such as mpId, Crew table's attributes and roleId (corresponding to "actor") to a python function "addActor" to execute INSERT statements in Crew, MotionPitcure and RoleCrew using Python's cursor.

In [ ]:
# crew_attribs: (crewId, personName, birthYear, deathYear, status)
def addActor(mpId, crewAttribs, roleId = 1):

    query4crew = """INSERT into Crew (crewId, personName, birthYear, deathYear, status) values (%s, %s, %s, %s, %s)"""

    query4mpc = """INSERT into MotionPictureCrew (mpId, crewId) values (%s, %s);"""

    query4rc = """INSERT into RoleCrew (roleId, crewId) values (%s, %s)"""

    cursor.execute(query4crew, crewAttribs)
    cnx.commit()
    cursor.execute(query4mpc, (mpId, crewAttribs[0]))
    cnx.commit()

#### Q10. deleteActor()
Deleting actor entry from the Crew table requires deletion of all the references from MotionPitcureCrew and RoleCrew. After this step we will remove the crewId reference from the Crew table.

In [ ]:
# crew_attribs: (crewId, personName, birthYear, deathYear, status)
def deleteActor(crewId, roleId = 1):

    query4mpc = """DELETE FROM MotionPictureCrew WHERE crewId = %s;"""
    
    query4crew = """DELETE FROM Crew WHERE crewId = %s;"""

    query4rc = """DELETE FROM RoleCrew WHERE crewId = %s and roleId = %s"""

    cursor.execute(query4mpc, crewId)
    cnx.commit()
    cursor.execute(query4crew, crewId)
    cnx.commit()
    cursor.execute(query4crew, crewId, roleId)
    cnx.commit()

#### Q11. Name and ages of all actors and their movies having above average rating
Here, we calculate average rating of all entries in Rating using AVG and then extract the names and ages of all actors and their movies having above average rating. The output has been obtained using MySQL Workbench using a dummy dataset.

In [ ]:
query = ('''
SELECT personName,name as role, age, count(*) as top_movies from Crew c
inner join RoleCrew rc on rc.rcId = c.crewId
inner join Role r on r.roleId = rc.rcId
inner join MotionPictureCrew mpc on mpc.mpcId = c.crewId
inner join MotionPicture mp on mp.mpId = mpc.mpcId
inner join rating rt on rt.mpId = mp.mpId
where rt.averageRating > (select avg(averageRating) from rating) and c.nuMovies > 2
group by c.crewId
having r.name = 'actress' or r.name = 'actor';''')
cursor.execute(query)
cursor.fetchone()

<img src="images/q11.png">

#### Q12. Indexing
We have added indexes in the database to speed-up lookup.

In [5]:
import time

In [ ]:
start=time.time()
query=('''select * from crew c where c.personName = 'Fred Astaire';''')
cursor.execute(query)
cursor.fetchone()
end=time.time()
timeWOIndex=end-start
print ("Time taken without index :",timeWOIndex)

In [ ]:
query=('''CREATE UNIQUE INDEX idx_crew_name ON crew (personName);;''')
cursor.execute(query)

In [ ]:
start=time.time()
query=('''select * from crew c where c.personName = 'Fred Astaire';''')
cursor.execute(query)
cursor.fetchone()
end=time.time()
timeWITHIndex=end-start
print ("Time taken with index :",timeWITHIndex)

Please find the execution time for before and after indexing below:

<img src="images/Q12.png">